# PyTorch Assignment: Multi-Layer Perceptron (MLP)

**[Duke Community Standard](http://integrity.duke.edu/standard.html): By typing your name below, you are certifying that you have adhered to the Duke Community Standard in completing this assignment.**

Name: David Zhu

### Multi-Layer Perceptrons

The simple logistic regression example we went over in the previous notebook is essentially a one-layer neural network, projecting straight from the input to the output predictions.
While this can be effective for linearly separable data, occasionally a little more complexity is necessary.
Neural networks with additional layers are typically able to learn more complex functions, leading to better performance.
These additional layers (called "hidden" layers) transform the input into one or more intermediate representations before making a final prediction.

In the logistic regression example, the way we performed the transformation was with a fully-connected layer, which consisted of a linear transform (matrix multiply plus a bias).
A neural network consisting of multiple successive fully-connected layers is commonly called a Multi-Layer Perceptron (MLP). 
In the simple MLP below, a 4-d input is projected to a 5-d hidden representation, which is then projected to a single output that is used to make the final prediction.

<img src="Figures/MLP.png" width="300"/>

For the assignment, you will be building a MLP for MNIST.
Mechanically, this is done very similary to our logistic regression example, but instead of going straight to a 10-d vector representing our output predictions, we might first transform to a 500-d vector with a "hidden" layer, then to the output of dimension 10.
Before you do so, however, there's one more important thing to consider.

### Nonlinearities

We typically include nonlinearities between layers of a neural network.
There's a number of reasons to do so.
For one, without anything nonlinear between them, successive linear transforms (fully connected layers) collapse into a single linear transform, which means the model isn't any more expressive than a single layer.
On the other hand, intermediate nonlinearities prevent this collapse, allowing neural networks to approximate more complex functions.

There are a number of nonlinearities commonly used in neural networks, but one of the most popular is the [rectified linear unit (ReLU)](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)):

\begin{align}
x = \max(0,x)
\end{align}

There are a number of ways to implement this in PyTorch.
We could do it with elementary PyTorch operations:

In [1]:
import torch

x = torch.rand(5, 3)*2 - 1
x_relu_max = torch.max(torch.zeros_like(x),x)

print("x: {}".format(x))
print("x after ReLU with max: {}".format(x_relu_max))

x: tensor([[ 0.2909,  0.6438, -0.8131],
        [ 0.8607, -0.6073,  0.3046],
        [-0.2574, -0.5535, -0.5443],
        [ 0.6783, -0.6375,  0.5562],
        [-0.8155, -0.6779,  0.1667]])
x after ReLU with max: tensor([[0.2909, 0.6438, 0.0000],
        [0.8607, 0.0000, 0.3046],
        [0.0000, 0.0000, 0.0000],
        [0.6783, 0.0000, 0.5562],
        [0.0000, 0.0000, 0.1667]])


Of course, PyTorch also has the ReLU implemented, for example in `torch.nn.functional`:

In [2]:
import torch.nn.functional as F

x_relu_F = F.relu(x)

print("x after ReLU with nn.functional: {}".format(x_relu_F))

x after ReLU with nn.functional: tensor([[0.2909, 0.6438, 0.0000],
        [0.8607, 0.0000, 0.3046],
        [0.0000, 0.0000, 0.0000],
        [0.6783, 0.0000, 0.5562],
        [0.0000, 0.0000, 0.1667]])


Same result.

### Assignment

Build a 2-layer MLP for MNIST digit classfication. Feel free to play around with the model architecture and see how the training time/performance changes, but to begin, try the following:

Image (784 dimensions) ->  
fully connected layer (500 hidden units) -> nonlinearity (ReLU) ->  
fully connected (10 hidden units) -> softmax

Try building the model both with basic PyTorch operations, and then again with more object-oriented higher-level APIs. 
You should get similar results!


*Some hints*:
- Even as we add additional layers, we still only require a single optimizer to learn the parameters.
Just make sure to pass all parameters to it!
- As you'll calculate in the Short Answer, this MLP model has many more parameters than the logisitic regression example, which makes it more challenging to learn.
To get the best performance, you may want to play with the learning rate and increase the number of training epochs.
- Be careful using `torch.nn.CrossEntropyLoss()`. 
If you look at the [PyTorch documentation](https://pytorch.org/docs/stable/nn.html#crossentropyloss): you'll see that `torch.nn.CrossEntropyLoss()` combines the softmax operation with the cross-entropy.
This means you need to pass in the logits (predictions pre-softmax) to this loss.
Computing the softmax separately and feeding the result into `torch.nn.CrossEntropyLoss()` will significantly degrade your model's performance!

In [4]:
### YOUR CODE HERE
# version: with basic PyTorch operations
import numpy as np
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from tqdm.notebook import tqdm


mnist_train = datasets.MNIST(root="./datasets", train=True, transform=transforms.ToTensor(), download=True)
mnist_test = datasets.MNIST(root="./datasets", train=False, transform=transforms.ToTensor(), download=True)

train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=100, shuffle=False)

## Training
# H1 = X * W1 + b1 -> rH1=ReLU(H1)
# Y = rH1 * W + b
# Initialize parameters, Hidden layer W1,b1,
W1 = torch.randn(784, 500)/np.sqrt(784)
W1.requires_grad_()
b1 = torch.zeros(500, requires_grad=True)
# final layer parameters, W, b
W = torch.randn(500, 10)/np.sqrt(500)
W.requires_grad_()
b = torch.zeros(10, requires_grad=True)
# Optimizer
optimizer = torch.optim.SGD([W1,b1,W,b], lr=0.1)

epochs=10

for epoch in range(epochs):
    # monitor training loss
    train_loss = 0.0
    
    ###################
    # train the model #
    ###################
    
    # Iterate through train set minibatchs
    for images, labels in tqdm(train_loader):
        # Zero out the gradients
        optimizer.zero_grad()
        
        # Forward pass
        x = images.view(-1, 28*28)
        # hidden layer 1
        h1 = torch.matmul(x, W1) + b1
        #nonlinearity (ReLU)
        h1_relu_F = F.relu(h1)
        # final layer
        y = torch.matmul(h1_relu_F, W) + b
        
        # loss
        cross_entropy = F.cross_entropy(y, labels)
        
        # Backward pass
        cross_entropy.backward()
        optimizer.step()
        
        # update running training loss
        train_loss += cross_entropy.item()*images.size(0)
        
    # print training statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch+1, 
        train_loss
        ))        
    
#100% 600/600 [04:17<00:00, 2.33it/s]
#Epoch: 1 	Training Loss: 0.438559

#100% 600/600 [02:45<00:00, 3.62it/s]
#Epoch: 2 	Training Loss: 0.237270

#100% 600/600 [05:26<00:00, 1.84it/s]
#Epoch: 3 	Training Loss: 0.184511

#100% 600/600 [04:00<00:00, 2.49it/s]
#Epoch: 4 	Training Loss: 0.151191

#100% 600/600 [02:40<00:00, 3.74it/s]
#Epoch: 5 	Training Loss: 0.128447

#100% 600/600 [01:21<00:00, 7.37it/s]
#Epoch: 6 	Training Loss: 0.111132

#100% 600/600 [01:35<00:00, 6.29it/s]
#Epoch: 7 	Training Loss: 0.098016

#100% 600/600 [01:22<00:00, 7.23it/s]
#Epoch: 8 	Training Loss: 0.087551

#100% 600/600 [01:20<00:00, 7.48it/s]
#Epoch: 9 	Training Loss: 0.078831

#100% 600/600 [01:23<00:00, 7.14it/s]
#Epoch: 10 	Training Loss: 0.071020

## Testing
correct = 0
total = len(mnist_test)

with torch.no_grad():
    # Iterate through test set minibatchs 
    for images, labels in tqdm(test_loader):
        # Forward pass
        x = images.view(-1, 28*28)
        h = torch.matmul(x, W1) + b1
        h_relu=F.relu(h)
        y = torch.matmul(h_relu, W) + b
        
        predictions = torch.argmax(y, dim=1)
        correct += torch.sum((predictions == labels).float())
    
print('Test accuracy: {}'.format(correct/total))










# Make sure to print out your accuracy on the test set at the end.


Epoch: 1 	Training Loss: 0.438797



Epoch: 2 	Training Loss: 0.235155



Epoch: 3 	Training Loss: 0.181731



Epoch: 4 	Training Loss: 0.148207



Epoch: 5 	Training Loss: 0.125350


...


Epoch: 7 	Training Loss: 0.095080



Epoch: 8 	Training Loss: 0.084738



Epoch: 9 	Training Loss: 0.076283



Epoch: 10 	Training Loss: 0.068898



Test accuracy: 0.9745000004768372


In [5]:
### YOUR CODE HERE
# version: with with more object-oriented higher-level APIs
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

class MNIST_MLP1(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(784, 500)
        #hidden layer
        self.lin2 = nn.Linear(500, 10)

    def forward(self, x):
        xx = F.relu(self.lin1(x))
        return self.lin2(xx)

# Load the data
mnist_train = datasets.MNIST(root="./datasets", train=True, transform=transforms.ToTensor(), download=True)
mnist_test = datasets.MNIST(root="./datasets", train=False, transform=transforms.ToTensor(), download=True)
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=100, shuffle=False)

## Training
# Instantiate model
model = MNIST_MLP1()  

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

epochs=10

for epoch in range(epochs):
    # monitor training loss
    train_loss = 0.0
    
    ###################
    # train the model #
    ###################
    
    # Iterate through train set minibatchs
    for images, labels in tqdm(train_loader):
        # Zero out the gradients
        optimizer.zero_grad()
        
        # Forward pass
        x = images.view(-1, 28*28)
        y = model(x)
        loss = criterion(y, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
    # print training statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch+1, 
        train_loss
        ))        
    
## Testing
correct = 0
total = len(mnist_test)

with torch.no_grad():
    # Iterate through test set minibatchs 
    for images, labels in tqdm(test_loader):
        # Forward pass
        x = images.view(-1, 28*28)
        y = model(x)
        
        predictions = torch.argmax(y, dim=1)
        correct += torch.sum((predictions == labels).float())
    
print('Test accuracy: {}'.format(correct/total))
    
    



Test accuracy: 0.9745000004768372


### Short answer
How many trainable parameters does your model have? 
How does this compare to the logisitic regression example?

In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print('MLP with hidden layer of 50-d parameters count is {}'.format(count_parameters(model)))

MLP with hidden layer of 50-d parameters count is 397510


`[Your answer here: W1+b1 + W+b : 784*500+500+500*10+10 = 392500+5010 = 397510 Logistic regression is 784*10 + 10 = 7850]`